### Extension Causal ML analysis

Since we got poor results during our initial Causal ML analysis, we tried a different approach to try to ameliorate the score:
- bellow you will see updated treatment
- Multiple sampling method to reduce class imbalance while keeping integrity of the fraud distribution
- X, T and S learner analysis
- Shap graph for features importance

#### Pre-processing

In [1]:
#importing the packages
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import xgboost as xgb
from xgboost import XGBClassifier

In [2]:
df = pd.read_csv('prepared_data2.csv')

In [3]:
df.columns

Index(['amount', 'Target', 'current_age', 'per_capita_income', 'yearly_income',
       'total_debt', 'credit_score', 'num_credit_cards',
       'use_chip_Chip Transaction', 'use_chip_Online Transaction',
       'merchant_state_CA', 'merchant_state_FL', 'merchant_state_IL',
       'merchant_state_MI', 'merchant_state_NC', 'merchant_state_OH',
       'merchant_state_ONLINE', 'merchant_state_Other', 'merchant_state_PA',
       'merchant_state_TX', 'gender_male', 'merch_risk_score',
       'merch_total_txn', 'avg_txn_merch', 'time_since_last_txn', 'weekend',
       'daily_transaction_count', 'weekly_transaction_count',
       'amount_change_rate', 'cumulative_amount', 'cumulative_transactions',
       'prev_time_since_txn', 'prev_merchant_risk_score',
       'txn_time_diff_change', 'merchant_risk_score_change',
       'large_amount_change', 'large_txn_time_diff_change',
       'large_merchant_risk_change', 'amt_income_ratio', 'debt_income_ratio',
       'client_fraud_rate', 'amt_avg_ratio'

#### Modeling

##### Selecting New Treatement that are "treatable"

We selected these three treatments—high transaction frequency, rapid spending increase, and suspicious activity/merchant risk change—because they capture different dimensions of transactional behavior that can serve as early warning signals for fraud. High transaction frequency is chosen because an unusually high number of transactions in a short period may indicate automated or unauthorized activity, suggesting that interventions such as temporary transaction limits or real-time alerts could be implemented to mitigate risk. Rapid spending increase was selected because a sudden surge in the amount spent, especially when it deviates significantly from a customer’s historical spending patterns, may be indicative of compromised accounts or fraudulent activity; this signal can prompt measures like enhanced verification protocols or spending reviews. Lastly, suspicious activity or significant changes in merchant risk scores are incorporated as they aggregate various qualitative and quantitative factors—from abnormal patterns in merchant behavior to irregularities flagged by anomaly detection systems—thereby offering a broader indicator of risk that can trigger manual investigations or targeted customer outreach. Collectively, these treatments align with our project’s objective of proactively identifying potential fraud, and they represent actionable points where targeted interventions (such as account freezes, transaction alerts, or additional verification steps) can be deployed to reduce financial risk.

#### Base Model:

In [12]:
# This script performs causal machine learning analysis using EconML.
# It includes:
# 1. Data Sampling: A small subset of the data is used to streamline the computation.
# 2. Definition of Treatments: Three treatment variables are defined -
#    a) High Transaction Frequency
#    b) Rapid Spending Increase
#    c) Suspicious Activity / Merchant Risk Change
# 3. Outcome and Confounders: Specifies the response variable and potential confounders.
# 4. Setup of Causal Learners: S-Learner, T-Learner, and X-Learner are applied with Linear Regression and XGBoost as base models.
# 5. Estimation of Treatment Effects: Computes ATE (Average Treatment Effect) and CATE (Conditional Average Treatment Effect).
#6. We use a sample of the data to allow for testing and it is adjustable
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression as LRSRegressor
import xgboost as xgb
XGBTRegressor = xgb.XGBRegressor
from econml.metalearners import SLearner, TLearner, XLearner

df_small = df.sample(n=1000, random_state=42)

daily_threshold = 10
weekly_threshold = 30
df_small['treatment_freq'] = ((df_small['daily_transaction_count'] > daily_threshold) |
                              (df_small['weekly_transaction_count'] > weekly_threshold)).astype(int)

amount_change_threshold = 1.5
df_small['treatment_rapid_spend'] = ((df_small['amount_change_rate'] > amount_change_threshold) |
                                     (df_small['large_amount_change'] == 1)).astype(int)

merchant_risk_threshold = 0.3
df_small['treatment_suspicious'] = ((df_small['suspicious_indiv_activity'] == 1) |
                                    (df_small['merchant_risk_score_change'] > merchant_risk_threshold)).astype(int)

treatment_cols = ['treatment_freq', 'treatment_rapid_spend', 'treatment_suspicious']

y = df_small['Target']

X = df_small.drop(columns=['Target'] + treatment_cols)

learners = {}

for treat in treatment_cols:
    T = df_small[treat]
    learners[treat] = {}

    s_lr = SLearner(overall_model=LRSRegressor())
    s_lr.fit(y, T, X=X)

    s_xgb = SLearner(overall_model=XGBTRegressor(random_state=42))
    s_xgb.fit(y, T, X=X)

    learners[treat]['S_LRS'] = s_lr
    learners[treat]['S_XGBT'] = s_xgb

    t_lr = TLearner(models=[LRSRegressor(), LRSRegressor()])
    t_lr.fit(y, T, X=X)

    t_xgb = TLearner(models=[XGBTRegressor(random_state=42), XGBTRegressor(random_state=42)])
    t_xgb.fit(y, T, X=X)

    learners[treat]['T_LRS'] = t_lr
    learners[treat]['T_XGBT'] = t_xgb

    x_lr = XLearner(models=LRSRegressor())
    x_lr.fit(y, T, X=X)

    x_xgb = XLearner(models=XGBTRegressor(random_state=42))
    x_xgb.fit(y, T, X=X)

    learners[treat]['X_LRS'] = x_lr
    learners[treat]['X_XGBT'] = x_xgb

new_X = X.copy()
results = {}

print("Estimated Treatment Effects (ATE and first 5 CATEs):\n")
for treat in treatment_cols:
    results[treat] = {}
    print(f"\nResults for treatment: {treat}")
    for model_name, model in learners[treat].items():
        cate_estimates = model.effect(new_X)
        ate_estimate = np.mean(cate_estimates)

        results[treat][model_name] = {
            'ATE': ate_estimate,
            'CATE_first_5': cate_estimates[:5]
        }

        print(f"{model_name}:")
        print(f"  ATE: {ate_estimate:.4f}")
        print(f"  CATE (first 5): {cate_estimates[:5]}")

Estimated Treatment Effects (ATE and first 5 CATEs):


Results for treatment: treatment_freq
S_LRS:
  ATE: 0.0089
  CATE (first 5): [0.00887915 0.00887915 0.00887915 0.00887915 0.00887915]
S_XGBT:
  ATE: 0.0000
  CATE (first 5): [0. 0. 0. 0. 0.]
T_LRS:
  ATE: 0.0020
  CATE (first 5): [ 0.01120655 -0.0027439   0.03427999  0.00658482 -0.00534301]
T_XGBT:
  ATE: 0.0020
  CATE (first 5): [ 1.11527061e-05 -8.39112909e-05  1.11527061e-05  1.11527061e-05
  9.78377284e-06]
X_LRS:
  ATE: 0.0071
  CATE (first 5): [ 2.17346864e-03 -7.43555790e-05  2.21638762e-02  8.36102527e-03
  6.76109304e-04]
X_XGBT:
  ATE: 0.0000
  CATE (first 5): [1.119645e-05 1.119645e-05 1.119645e-05 1.119645e-05 1.119645e-05]

Results for treatment: treatment_rapid_spend
S_LRS:
  ATE: -0.0053
  CATE (first 5): [-0.00534653 -0.00534653 -0.00534653 -0.00534653 -0.00534653]
S_XGBT:
  ATE: 0.0000
  CATE (first 5): [0. 0. 0. 0. 0.]
T_LRS:
  ATE: -0.0036
  CATE (first 5): [-0.01323379  0.00324641 -0.06916176 -0.00680782  0.0104

#### Using Standard Scaler  and Logistic Regression

In [11]:
# 1. Define Treatment Variables on df_small
# - Treatment 1: High Transaction Frequency
# - Treatment 2: Rapid Spending Increase
# - Treatment 3: Suspicious Activity / Merchant Risk Change
# 2. Define Outcome and Confounders, then Scale the Confounders
# 3. Set up a logistic regression propensity learner with increased max_iter.
# 4. Set Up Causal Learners using EconML with LRSRegressor and XGBTRegressor
# 5. Estimate ATE and CATE

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression as LRSRegressor
import xgboost as xgb
XGBTRegressor = xgb.XGBRegressor
from econml.metalearners import SLearner, TLearner, XLearner
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

df_small = df.sample(n=1000, random_state=42)

daily_threshold = 10
weekly_threshold = 30
df_small['treatment_freq'] = ((df_small['daily_transaction_count'] > daily_threshold) |
                              (df_small['weekly_transaction_count'] > weekly_threshold)).astype(int)

amount_change_threshold = 1.5
df_small['treatment_rapid_spend'] = ((df_small['amount_change_rate'] > amount_change_threshold) |
                                     (df_small['large_amount_change'] == 1)).astype(int)

merchant_risk_threshold = 0.3
df_small['treatment_suspicious'] = ((df_small['suspicious_indiv_activity'] == 1) |
                                    (df_small['merchant_risk_score_change'] > merchant_risk_threshold)).astype(int)

treatment_cols = ['treatment_freq', 'treatment_rapid_spend', 'treatment_suspicious']

y = df_small['Target']

X = df_small.drop(columns=['Target'] + treatment_cols)

scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

propensity_model = LogisticRegression(max_iter=1000, solver='lbfgs')

learners = {}

for treat in treatment_cols:
    T = df_small[treat]
    learners[treat] = {}

    s_lr = SLearner(overall_model=LRSRegressor())
    s_lr.fit(y, T, X=X_scaled)

    s_xgb = SLearner(overall_model=XGBTRegressor(random_state=42))
    s_xgb.fit(y, T, X=X_scaled)

    learners[treat]['S_LRS'] = s_lr
    learners[treat]['S_XGBT'] = s_xgb

    t_lr = TLearner(models=[LRSRegressor(), LRSRegressor()])
    t_lr.fit(y, T, X=X_scaled)

    t_xgb = TLearner(models=[XGBTRegressor(random_state=42), XGBTRegressor(random_state=42)])
    t_xgb.fit(y, T, X=X_scaled)

    learners[treat]['T_LRS'] = t_lr
    learners[treat]['T_XGBT'] = t_xgb

    x_lr = XLearner(models=LRSRegressor(), propensity_model=propensity_model)
    x_lr.fit(y, T, X=X_scaled)

    x_xgb = XLearner(models=XGBTRegressor(random_state=42), propensity_model=propensity_model)
    x_xgb.fit(y, T, X=X_scaled)

    learners[treat]['X_LRS'] = x_lr
    learners[treat]['X_XGBT'] = x_xgb

new_X = X_scaled.copy()
results = {}

print("Estimated Treatment Effects (ATE and first 5 CATEs):\n")
for treat in treatment_cols:
    results[treat] = {}
    print(f"\nResults for treatment: {treat}")
    for model_name, model in learners[treat].items():
        cate_estimates = model.effect(new_X)
        ate_estimate = np.mean(cate_estimates)

        results[treat][model_name] = {
            'ATE': ate_estimate,
            'CATE_first_5': cate_estimates[:5]
        }

        print(f"{model_name}:")
        print(f"  ATE: {ate_estimate:.4f}")
        print(f"  CATE (first 5): {cate_estimates[:5]}")

Estimated Treatment Effects (ATE and first 5 CATEs):


Results for treatment: treatment_freq
S_LRS:
  ATE: 0.0042
  CATE (first 5): [0.00421143 0.00427246 0.00427246 0.00424194 0.00421143]
S_XGBT:
  ATE: 0.0000
  CATE (first 5): [0. 0. 0. 0. 0.]
T_LRS:
  ATE: 0.0022
  CATE (first 5): [ 0.01215799 -0.00321731  0.03701279  0.00367966 -0.01145706]
T_XGBT:
  ATE: 0.0020
  CATE (first 5): [ 1.11527061e-05 -8.39112909e-05  1.11527061e-05  1.11527061e-05
  9.78377284e-06]
X_LRS:
  ATE: 0.0094
  CATE (first 5): [0.01073324 0.00560032 0.01586046 0.00971487 0.01022724]
X_XGBT:
  ATE: 0.0000
  CATE (first 5): [1.11964498e-05 1.11913817e-05 1.11964492e-05 1.11964496e-05
 1.11962242e-05]

Results for treatment: treatment_rapid_spend
S_LRS:
  ATE: -0.0042
  CATE (first 5): [-0.00415039 -0.00415039 -0.00415039 -0.00415039 -0.00415039]
S_XGBT:
  ATE: 0.0000
  CATE (first 5): [0. 0. 0. 0. 0.]
T_LRS:
  ATE: -94774798898.7243
  CATE (first 5): [ 1.49383545e-02 -6.25610352e-04 -9.87237489e+11 -7.76672363e

#### Using RandomOverSampler

In [13]:
# 1. Treatment variables are defined based on transaction frequency, rapid spending
#    increase, and suspicious activity or merchant risk change.
# 2. Outcome variable (Target) and confounders are defined. The confounders are scaled
#    using `StandardScaler`.
# 3. Multiple causal learners (S-Learner, T-Learner, and X-Learner) are trained for
#    each treatment variable after addressing class imbalance using `RandomOverSampler`.

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression as LRSRegressor
import xgboost as xgb
XGBTRegressor = xgb.XGBRegressor
from econml.metalearners import SLearner, TLearner, XLearner
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import RandomOverSampler

df_small = df.sample(n=1000, random_state=42)
daily_threshold = 10
weekly_threshold = 30
df_small['treatment_freq'] = ((df_small['daily_transaction_count'] > daily_threshold) |
                              (df_small['weekly_transaction_count'] > weekly_threshold)).astype(int)
amount_change_threshold = 1.5
df_small['treatment_rapid_spend'] = ((df_small['amount_change_rate'] > amount_change_threshold) |
                                     (df_small['large_amount_change'] == 1)).astype(int)
merchant_risk_threshold = 0.3
df_small['treatment_suspicious'] = ((df_small['suspicious_indiv_activity'] == 1) |
                                    (df_small['merchant_risk_score_change'] > merchant_risk_threshold)).astype(int)
treatment_cols = ['treatment_freq', 'treatment_rapid_spend', 'treatment_suspicious']
y = df_small['Target']
X = df_small.drop(columns=['Target'] + treatment_cols)
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
propensity_model = LogisticRegression(max_iter=1000, solver='lbfgs')
learners = {}
results = {}

for treat in treatment_cols:
    T = df_small[treat]
    df_temp = X_scaled.copy()
    df_temp['T'] = T.values
    df_temp['y'] = y.values
    ros = RandomOverSampler(random_state=42)
    df_resampled, _ = ros.fit_resample(df_temp, df_temp['T'])
    X_res = df_resampled.drop(columns=['T', 'y'])
    T_res = df_resampled['T']
    y_res = df_resampled['y']
    learners[treat] = {}
    s_lr = SLearner(overall_model=LRSRegressor())
    s_lr.fit(y_res, T_res, X=X_res)
    s_xgb = SLearner(overall_model=XGBTRegressor(random_state=42))
    s_xgb.fit(y_res, T_res, X=X_res)
    learners[treat]['S_LRS'] = s_lr
    learners[treat]['S_XGBT'] = s_xgb
    t_lr = TLearner(models=[LRSRegressor(), LRSRegressor()])
    t_lr.fit(y_res, T_res, X=X_res)
    t_xgb = TLearner(models=[XGBTRegressor(random_state=42), XGBTRegressor(random_state=42)])
    t_xgb.fit(y_res, T_res, X=X_res)
    learners[treat]['T_LRS'] = t_lr
    learners[treat]['T_XGBT'] = t_xgb
    x_lr = XLearner(models=LRSRegressor(), propensity_model=propensity_model)
    x_lr.fit(y_res, T_res, X=X_res)
    x_xgb = XLearner(models=XGBTRegressor(random_state=42), propensity_model=propensity_model)
    x_xgb.fit(y_res, T_res, X=X_res)
    learners[treat]['X_LRS'] = x_lr
    learners[treat]['X_XGBT'] = x_xgb
    new_X = X_res.copy()
    results[treat] = {}
    print(f"\nResults for treatment: {treat}")
    for model_name, model in learners[treat].items():
        cate_estimates = model.effect(new_X)
        ate_estimate = np.mean(cate_estimates)
        results[treat][model_name] = {
            'ATE': ate_estimate,
            'CATE_first_5': cate_estimates[:5]
        }
        print(f"{model_name}:")
        print(f"  ATE: {ate_estimate:.4f}")
        print(f"  CATE (first 5): {cate_estimates[:5]}")


Results for treatment: treatment_freq
S_LRS:
  ATE: 0.0073
  CATE (first 5): [0.00730896 0.00726318 0.0072937  0.00730133 0.00730133]
S_XGBT:
  ATE: 0.0000
  CATE (first 5): [0. 0. 0. 0. 0.]
T_LRS:
  ATE: 0.0067
  CATE (first 5): [ 0.01215799 -0.00321731  0.03701279  0.00367966 -0.01145706]
T_XGBT:
  ATE: 0.0010
  CATE (first 5): [ 1.11527061e-05 -8.39112909e-05  1.11527061e-05  1.11527061e-05
  9.78377284e-06]
X_LRS:
  ATE: 0.0090
  CATE (first 5): [ 0.00601891 -0.00110475 -0.0007485   0.00646459  0.01338162]
X_XGBT:
  ATE: 0.0000
  CATE (first 5): [1.11994332e-05 1.11991843e-05 1.11994332e-05 1.11994332e-05
 1.11994155e-05]

Results for treatment: treatment_rapid_spend
S_LRS:
  ATE: -0.0050
  CATE (first 5): [-0.00498199 -0.00497437 -0.00500488 -0.00498962 -0.00497437]
S_XGBT:
  ATE: 0.0000
  CATE (first 5): [0. 0. 0. 0. 0.]
T_LRS:
  ATE: -302407669310.8331
  CATE (first 5): [ 1.49383545e-02 -6.25610352e-04 -9.87237489e+11 -7.76672363e-03
 -1.98516846e-02]
T_XGBT:
  ATE: -0.0012
  C

#### USING ADASYN and Ridge Regressor

In [19]:
# 1. Preparing data for modeling (scaling, creating treatments, and resampling imbalanced data).
# 2. Training meta-learners for each treatment using different models (Ridge Regressor and XGBoost Regressor).
# 3. Evaluating the average treatment effect (ATE) and conditional average treatment effects (CATEs).

import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from econml.metalearners import SLearner, TLearner, XLearner
from imblearn.over_sampling import ADASYN
from xgboost import XGBRegressor

RRSRegressor = Ridge()
XGBTRegressor = XGBRegressor

df_small = df.sample(n=600000, random_state=42)

daily_threshold = 10
weekly_threshold = 30
amount_change_threshold = 1.5
merchant_risk_threshold = 0.3

df_small['treatment_freq'] = ((df_small['daily_transaction_count'] > daily_threshold) |
                              (df_small['weekly_transaction_count'] > weekly_threshold)).astype(int)

df_small['treatment_rapid_spend'] = ((df_small['amount_change_rate'] > amount_change_threshold) |
                                     (df_small['large_amount_change'] == 1)).astype(int)

df_small['treatment_suspicious'] = ((df_small['suspicious_indiv_activity'] == 1) |
                                    (df_small['merchant_risk_score_change'] > merchant_risk_threshold)).astype(int)

treatment_cols = ['treatment_freq', 'treatment_rapid_spend', 'treatment_suspicious']
y = df_small['Target']
X = df_small.drop(columns=['Target'] + treatment_cols)

scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

propensity_model = LogisticRegression(max_iter=1000, solver='lbfgs')

learners = {}
results = {}

for treat in treatment_cols:
    T = df_small[treat]
    df_temp = X_scaled.copy()
    df_temp['T'] = T.values
    df_temp['y'] = y.values

    adasyn = ADASYN(random_state=42, n_neighbors=min(5, df_temp['T'].value_counts().min() - 1))
    df_resampled, _ = adasyn.fit_resample(df_temp, df_temp['T'])

    X_res = df_resampled.drop(columns=['T', 'y'])
    T_res = df_resampled['T']
    y_res = df_resampled['y']

    learners[treat] = {}

    s_rrs = SLearner(overall_model=RRSRegressor)
    s_rrs.fit(y_res, T_res, X=X_res)
    learners[treat]['S_RRS'] = s_rrs

    s_xgb = SLearner(overall_model=XGBTRegressor(random_state=42))
    s_xgb.fit(y_res, T_res, X=X_res)
    learners[treat]['S_XGBT'] = s_xgb

    t_rrs = TLearner(models=[RRSRegressor, RRSRegressor])
    t_rrs.fit(y_res, T_res, X=X_res)
    learners[treat]['T_RRS'] = t_rrs

    t_xgb = TLearner(models=[XGBTRegressor(random_state=42), XGBTRegressor(random_state=42)])
    t_xgb.fit(y_res, T_res, X=X_res)
    learners[treat]['T_XGBT'] = t_xgb

    x_rrs = XLearner(models=RRSRegressor, propensity_model=propensity_model)
    x_rrs.fit(y_res, T_res, X=X_res)
    learners[treat]['X_RRS'] = x_rrs

    x_xgb = XLearner(models=XGBTRegressor(random_state=42), propensity_model=propensity_model)
    x_xgb.fit(y_res, T_res, X=X_res)
    learners[treat]['X_XGBT'] = x_xgb

    new_X = X_res.copy()
    results[treat] = {}

    for model_name, model in learners[treat].items():
        cate_estimates = model.effect(new_X)
        ate_estimate = np.mean(cate_estimates)

        results[treat][model_name] = {
            'ATE': ate_estimate,
            'CATE_first_5': cate_estimates[:5]
        }

        print(f"\nResults for treatment: {treat}")
        print(f"{model_name}:")
        print(f"  ATE: {ate_estimate:.4f}")
        print(f"  CATE (first 5): {cate_estimates[:5]}")


Results for treatment: treatment_freq
S_RRS:
  ATE: 0.0032
  CATE (first 5): [0.00321089 0.00321089 0.00321089 0.00321089 0.00321089]

Results for treatment: treatment_freq
S_XGBT:
  ATE: 0.0009
  CATE (first 5): [3.49410716e-03 2.62449525e-04 1.15286303e-03 5.78663545e-04
 6.21874206e-05]

Results for treatment: treatment_freq
T_RRS:
  ATE: 0.0030
  CATE (first 5): [0.00510554 0.00137916 0.00990803 0.00028266 0.00332597]

Results for treatment: treatment_freq
T_XGBT:
  ATE: -0.0002
  CATE (first 5): [-9.50012065e-04 -7.45160069e-05 -2.96447705e-03  4.73147666e-04
 -8.01460701e-05]

Results for treatment: treatment_freq
X_RRS:
  ATE: 0.0030
  CATE (first 5): [0.00510802 0.00137468 0.00990731 0.00027842 0.00332205]

Results for treatment: treatment_freq
X_XGBT:
  ATE: 0.0056
  CATE (first 5): [ 2.80486979e-03 -8.01083893e-02 -7.99820991e-04 -5.77082865e-05
  6.94155227e-03]

Results for treatment: treatment_rapid_spend
S_RRS:
  ATE: -0.0008
  CATE (first 5): [-0.0007807 -0.0007807 -0.0

#### USING ROS, Hypermarameter tunning

In [13]:
# 1. Sample a subset of the dataset for faster runtime during experimentation.
# 2. Define treatment variables based on transactional behavior thresholds.
# 3. Define the outcome ('Target') and apply scaling to the features (confounders).
# 4. Perform hyperparameter tuning to find the best parameters for Ridge regression
#    and XGBoost models, which are used as base learners.
# 5. Use `RandomOverSampler` to balance treatment groups, ensuring no class imbalance
#    while fitting the models.
# 6. Train S-Learner, T-Learner, and X-Learner models for each defined treatment.
# 7. Estimate both the Average Treatment Effect (ATE) and Conditional Average Treatment Effects (CATE).
#    CATE reflects estimates for specific subsets of the data.
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import Ridge
from xgboost import XGBRegressor
from econml.metalearners import SLearner, TLearner, XLearner
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import RandomOverSampler

df_small = df.sample(n=600000, random_state=42)

daily_threshold = 10
weekly_threshold = 30
df_small['treatment_freq'] = ((df_small['daily_transaction_count'] > daily_threshold) |
                              (df_small['weekly_transaction_count'] > weekly_threshold)).astype(int)

amount_change_threshold = 1.5
df_small['treatment_rapid_spend'] = ((df_small['amount_change_rate'] > amount_change_threshold) |
                                     (df_small['large_amount_change'] == 1)).astype(int)

merchant_risk_threshold = 0.3
df_small['treatment_suspicious'] = ((df_small['suspicious_indiv_activity'] == 1) |
                                    (df_small['merchant_risk_score_change'] > merchant_risk_threshold)).astype(int)

treatment_cols = ['treatment_freq', 'treatment_rapid_spend', 'treatment_suspicious']

y = df_small['Target']
X = df_small.drop(columns=['Target'] + treatment_cols)
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

propensity_model = LogisticRegression(max_iter=1000, solver='lbfgs')

param_grid_ridge = {'alpha': [0.1, 1.0, 10.0, 100.0]}
grid_ridge = GridSearchCV(Ridge(), param_grid_ridge, cv=5, scoring='neg_mean_squared_error')
grid_ridge.fit(X_scaled, y)
best_alpha = grid_ridge.best_params_['alpha']
print("Best Ridge alpha:", best_alpha)

param_grid_xgb = {'n_estimators': [50, 100, 200],
                  'max_depth': [3, 5, 7],
                  'learning_rate': [0.01, 0.1, 0.2]}
grid_xgb = GridSearchCV(XGBRegressor(random_state=42, objective='reg:squarederror'),
                        param_grid_xgb, cv=5, scoring='neg_mean_squared_error')
grid_xgb.fit(X_scaled, y)
best_params_xgb = grid_xgb.best_params_
print("Best XGBoost params:", best_params_xgb)

def get_tuned_ridge():
    return Ridge(alpha=best_alpha)

def get_tuned_xgb():
    return XGBRegressor(random_state=42, objective='reg:squarederror', **best_params_xgb)

learners = {}
results = {}

for treat in treatment_cols:
    T = df_small[treat]
    df_temp = X_scaled.copy()
    df_temp['T'] = T.values
    df_temp['y'] = y.values

    ros = RandomOverSampler(random_state=42)
    df_resampled, _ = ros.fit_resample(df_temp, df_temp['T'])

    X_res = df_resampled.drop(columns=['T', 'y'])
    T_res = df_resampled['T']
    y_res = df_resampled['y']

    learners[treat] = {}

    s_lr = SLearner(overall_model=get_tuned_ridge())
    s_lr.fit(y_res, T_res, X=X_res)

    s_xgb = SLearner(overall_model=get_tuned_xgb())
    s_xgb.fit(y_res, T_res, X=X_res)

    learners[treat]['S_Ridge'] = s_lr
    learners[treat]['S_XGBT'] = s_xgb

    t_lr = TLearner(models=[get_tuned_ridge(), get_tuned_ridge()])
    t_lr.fit(y_res, T_res, X=X_res)

    t_xgb = TLearner(models=[get_tuned_xgb(), get_tuned_xgb()])
    t_xgb.fit(y_res, T_res, X=X_res)

    learners[treat]['T_Ridge'] = t_lr
    learners[treat]['T_XGBT'] = t_xgb

    x_lr = XLearner(models=get_tuned_ridge(), propensity_model=propensity_model)
    x_lr.fit(y_res, T_res, X=X_res)

    x_xgb = XLearner(models=get_tuned_xgb(), propensity_model=propensity_model)
    x_xgb.fit(y_res, T_res, X=X_res)

    learners[treat]['X_LRS'] = x_lr
    learners[treat]['X_XGBT'] = x_xgb

    new_X = X_res.copy()
    results[treat] = {}
    print(f"\nResults for treatment: {treat}")
    for model_name, model in learners[treat].items():
        cate_estimates = model.effect(new_X)
        ate_estimate = np.mean(cate_estimates)
        results[treat][model_name] = {'ATE': ate_estimate, 'CATE_first_5': cate_estimates[:5]}
        print(f"{model_name}:")
        print(f"  ATE: {ate_estimate:.4f}")
        print(f"  CATE (first 5): {cate_estimates[:5]}")

Best Ridge alpha: 0.1
Best XGBoost params: {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200}

Results for treatment: treatment_freq
S_Ridge:
  ATE: 0.0012
  CATE (first 5): [0.00123708 0.00123708 0.00123708 0.00123708 0.00123708]
S_XGBT:
  ATE: -0.0000
  CATE (first 5): [0. 0. 0. 0. 0.]
T_Ridge:
  ATE: 0.0299
  CATE (first 5): [0.0726579  0.02063734 0.03872644 0.10074011 0.03218214]
T_XGBT:
  ATE: -0.0020
  CATE (first 5): [ 3.10726394e-03 -2.06862052e-04  1.70826376e-03  6.52611707e-05
 -1.64885877e-03]
X_LRS:
  ATE: 0.0299
  CATE (first 5): [0.07265799 0.02063695 0.03872632 0.10073951 0.03218173]
X_XGBT:
  ATE: -0.0021
  CATE (first 5): [0.00468646 0.00539277 0.00157829 0.00166613 0.0022906 ]

Results for treatment: treatment_rapid_spend
S_Ridge:
  ATE: 0.0009
  CATE (first 5): [0.0008625 0.0008625 0.0008625 0.0008625 0.0008625]
S_XGBT:
  ATE: -0.0000
  CATE (first 5): [3.85283493e-05 4.25242979e-05 0.00000000e+00 0.00000000e+00
 0.00000000e+00]
T_Ridge:
  ATE: 0.0013
  CAT

#### Interpretation

Treatment: High Transaction Frequency: The estimated effects for high transaction frequency vary across models. The Ridge‐based S‑Learner indicates a very small positive effect (an average treatment effect, or ATE, of about 0.12%), suggesting that an increase in transaction frequency might be weakly associated with a slightly higher risk or outcome value. However, when using the T‑Learner and X‑Learner with Ridge, the ATE jumps to roughly 3%, indicating that under these modeling frameworks, high frequency appears to be more influential—but this is not corroborated by the XGBoost‑based models, which consistently report negligible or even slightly negative effects.

Treatment: Rapid Spending Increase: For rapid spending increase, the linear models (across S‑, T‑, and X‑Learners using Ridge) show very small positive effects (with ATEs in the range of about 0.07% to 0.13%), whereas the XGBoost‑based models again indicate nearly zero impact. The minimal effect sizes across all specifications imply that a sudden surge in spending is not, by itself, a strong driver of the target outcome. From a business perspective, this weak signal means that relying solely on rapid spending as an indicator may not be sufficient to justify strong remedial actions.

Treatment: Suspicious Activity / Merchant Risk Change: The results for suspicious activity exhibit the most significant variation. The linear models (S‑Learner and X‑Learner with Ridge) suggest a meaningful positive effect, with ATEs ranging from roughly 1.7% to 1.8%, and the T‑Learner indicating an even higher average effect of around 1%. More notably, one of the XGBoost‑based learners (the X‑Learner) estimates a considerably higher average effect (about 5.7%), with individual conditional effects (CATEs) reaching as high as 55% for some observations. This wide range points to substantial heterogeneity—indicating that while a subset of transactions flagged as suspicious shows a strong association with increased risk, others do not. This variability is critical from a business standpoint: it suggests that suspicious activity is a potentially powerful indicator, but its signal is inconsistent across the population.